In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import json
import pandas as pd
import json
import yahoo_fin.stock_info as si
import random
import pyttsx3
import numpy as np

In [2]:
# open list of user agents
with open("../../../Data/user_agent_list.json", "r") as f:
    user_agent_list = json.load(f)

In [3]:
# tickers
with open("../relevant_trading_212_tickers.json", "r") as f:
    ticker_list = json.load(f)

In [14]:
ticker_df_list = []
n = 0
m = 0

for ticker in ticker_list[3000:]:
    try:
        

        financial_data = si.get_financials(ticker, yearly=False)
        ebit = financial_data["quarterly_income_statement"].loc["ebit"]
        total_assets = financial_data["quarterly_balance_sheet"].loc["totalAssets"]
        stockholder_equity = financial_data["quarterly_balance_sheet"].loc["totalStockholderEquity"]
        share_rep = financial_data["quarterly_cash_flow"].loc["repurchaseOfStock"]

        ticker_df = pd.DataFrame({
            "ticker" : ticker,
            "ebit_date" : ebit.index,
            "ebit" : ebit.values,
            "ass_tot_date" : total_assets.index,
            "ass_tot" : total_assets.values,
            "book_equ_date" : stockholder_equity.index,
            "book_equ" : stockholder_equity.values,
            "share_rep_date" : share_rep.index,
            "share_rep" : share_rep.values
        })

        ticker_df_list.append(ticker_df)
        n += 1
        print("n: " + str(n))
    except:
        m += 1
        print("m: " + str(m))
        pass
    
text = "Ich bin fertig du Sneck"

speaker = pyttsx3.init()
speaker.say(text)
speaker.runAndWait()



n: 1
m: 1
m: 2
n: 2
m: 3
m: 4
m: 5
n: 3
m: 6
m: 7
n: 4
n: 5
m: 8
n: 6
n: 7
n: 8
n: 9
m: 9
m: 10
m: 11
m: 12
n: 10
m: 13
m: 14
m: 15
n: 11
m: 16
n: 12
n: 13
n: 14
m: 17
n: 15
m: 18
m: 19
n: 16
n: 17
m: 20
m: 21
n: 18
n: 19
m: 22
m: 23
m: 24
m: 25
n: 20
n: 21
n: 22
m: 26
n: 23
m: 27
n: 24
n: 25
m: 28
m: 29
m: 30
n: 26
n: 27
n: 28
n: 29
m: 31
n: 30
n: 31
m: 32
n: 32
m: 33
n: 33
m: 34
m: 35
n: 34
m: 36
n: 35
m: 37
m: 38
n: 36
m: 39
n: 37
m: 40
n: 38
m: 41
n: 39
m: 42
n: 40
n: 41
n: 42
n: 43
m: 43
n: 44
n: 45
m: 44
n: 46
m: 45
m: 46
n: 47
n: 48
m: 47
m: 48
n: 49
m: 49
m: 50
n: 50
m: 51
m: 52
n: 51
n: 52
m: 53
n: 53
m: 54
m: 55
n: 54
m: 56
m: 57
n: 55
n: 56
m: 58
n: 57
n: 58
m: 59
n: 59
n: 60
n: 61
m: 60
n: 62
n: 63
m: 61
n: 64
n: 65
n: 66
n: 67
n: 68
n: 69
n: 70
m: 62
m: 63
n: 71
n: 72
n: 73
n: 74
n: 75
m: 64
n: 76
m: 65
n: 77
n: 78
n: 79
m: 66
n: 80
m: 67
m: 68
m: 69
n: 81
n: 82
n: 83
m: 70
m: 71
n: 84
n: 85
m: 72
m: 73
m: 74
n: 86
n: 87
m: 75
n: 88
m: 76
n: 89
m: 77
n: 90
n: 91
m: 78
m: 7

In [15]:
all_ticker_df = pd.concat(ticker_df_list)
all_ticker_df.to_csv("all_ticker_funds_from_3000.csv")

In [4]:
bis_1000 = pd.read_csv("all_ticker_funds_0_1000.csv")
tau_bis_3000 = pd.read_csv("all_ticker_funds_1000_3000.csv")
drei_tau_bis_ende = pd.read_csv("all_ticker_funds_from_3000.csv")

In [5]:
dfs = [bis_1000, tau_bis_3000, drei_tau_bis_ende]
all_funds_df = pd.concat(dfs)

In [6]:
len(all_funds_df.ticker.unique())

2836

In [7]:
for e, a, b, s in zip(all_funds_df.ebit_date, all_funds_df.ass_tot_date, all_funds_df.book_equ_date, all_funds_df.share_rep_date):
    if e == a == b == s:
        pass
    else:
        print("nope")
        
# date is equal

In [8]:
funds_data = all_funds_df[["ebit_date", "ticker", "ass_tot", "book_equ", "share_rep"]]

In [9]:
funds_data.columns = ["date", "ticker", "assets", "book_equity", "shares_rep"]

In [10]:
funds_tickers = funds_data.ticker.unique()
funds_dates = funds_data.date.unique()

## Add shares outstanding to the df

In [11]:
shares_out_df = pd.read_csv("shares_outstanding_df.csv", index_col=0)

In [12]:
shares_out_df.head()

,date,ticker,shares_outstanding
0,"Nov. 6, 2020",EMCF,2709000.0
1,"Sept. 30, 2020",EMCF,2709000.0
2,"Aug. 11, 2020",EMCF,2709000.0
3,"June 30, 2020",EMCF,2709000.0
4,"May 12, 2020",EMCF,2709000.0


In [13]:
# get the date in following format:  yyyy-mm-dd
dates = shares_out_df.date.tolist()

# create dict
months_dict = {
    "Nov." : "-11-",
    "Sept." : "-09-",
    "Aug." : "-08-",
    "June" : "-06-",
    "May" : "-05-",
    "March" : "-03-",
    "April" : "-04-",
    "Feb." : "-02-",
    "Oct." : "-10-",
    "July" : "-07-", 
    "Dec." : "-12-",
    "Jan." : "-01-"
}

days_dict = {
    "1" : "01",
    "2" : "02",
    "3" : "03",
    "4": "04",
    "5" : "05",
    "6" : "06",
    "7" : "07",
    "8" : "08",
    "9" : "09"
}

In [14]:
splitted_date = [date.split(",") for date in shares_out_df.date.tolist()]

In [15]:
shares_out_df["month_day"] = [i[0] for i in splitted_date]
shares_out_df["year"] = [i[1] for i in splitted_date]

In [16]:
splitted_month = [date.split() for date in shares_out_df.month_day.tolist()]

In [17]:
shares_out_df["month"] = [i[0] for i in splitted_month]
shares_out_df["day"] = [i[1] for i in splitted_month]

In [18]:
shares_out_df.reset_index(drop = True, inplace = True)

In [19]:
shares_out_df["year"] = [i.strip() for i in shares_out_df["year"].tolist()]

In [20]:
shares_out_df.head()

,date,ticker,shares_outstanding,month_day,year,month,day
0,"Nov. 6, 2020",EMCF,2709000.0,Nov. 6,2020,Nov.,6
1,"Sept. 30, 2020",EMCF,2709000.0,Sept. 30,2020,Sept.,30
2,"Aug. 11, 2020",EMCF,2709000.0,Aug. 11,2020,Aug.,11
3,"June 30, 2020",EMCF,2709000.0,June 30,2020,June,30
4,"May 12, 2020",EMCF,2709000.0,May 12,2020,May,12


In [21]:
dates = []

for day, month, year in zip(shares_out_df.day, shares_out_df.month, shares_out_df.year):
    m = months_dict[month]
    if day in days_dict.keys():
        d = days_dict[day]
    else:
        d = day
    
    date = str(year + m + d)
    
    dates.append(date)

In [22]:
# add the correct date_column

shares_out_df["rel_date"] = dates

In [23]:
rel_so_df = shares_out_df[["rel_date", "ticker", "shares_outstanding"]]

In [24]:
common_tickers = set.intersection(set(funds_data.ticker), set(rel_so_df.ticker))
common_dates = set.intersection(set(funds_data.date), set(rel_so_df.rel_date))

In [25]:
rel_so_df = rel_so_df[(rel_so_df.rel_date.isin(common_dates)) & (rel_so_df.ticker.isin(common_tickers))]

In [26]:
rel_so_df.reset_index(drop = True, inplace = True)
len(rel_so_df)

24862

In [27]:
funds_data.reset_index(drop = True, inplace = True)

In [28]:
# list of the shares outstanding at the relevant date

s_o_list = []

for d, t in zip(funds_data.date, funds_data.ticker):
    try:
        s_o = rel_so_df[(rel_so_df.ticker == t) & (rel_so_df.rel_date == d)]["shares_outstanding"].item()
        s_o_list.append(s_o)
       
    except:
        s_o_list.append(np.nan)
    

In [29]:
funds_data["shares_outstanding"] = s_o_list

C:\Users\Leonhard\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
funds_data

,date,ticker,assets,book_equity,shares_rep,shares_outstanding
0,2020-09-30,ARCC,1.495000e+10,6.963000e+09,NaN,422620000.0
1,2020-06-30,ARCC,1.451700e+10,6.691000e+09,NaN,423000000.0
2,2020-03-31,ARCC,1.580600e+10,6.583000e+09,-100000000.0,423000000.0
3,2019-12-31,ARCC,1.490500e+10,7.467000e+09,-100000000.0,431000000.0
4,2020-09-30,AEIS,1.642519e+09,7.681520e+08,-4331000.0,38270000.0
...,...,...,...,...,...,...
11339,2019-12-31,PRSP,6.189000e+09,2.206000e+09,-16000000.0,161530000.0
11340,2020-09-30,WTRG,1.339949e+10,4.635753e+09,-2000.0,245270000.0
11341,2020-06-30,WTRG,1.302841e+10,4.635534e+09,-4000.0,245150000.0
11342,2020-03-31,WTRG,1.292899e+10,4.613163e+09,-4339000.0,245040000.0


In [31]:
# drop data when shares outstanding is missing
rel_funds = funds_data.dropna(subset=["shares_outstanding"])

In [32]:
rel_funds.reset_index(drop = True, inplace = True)

In [33]:
# import the price data
price_data = pd.read_csv("../relevant_price_data_from_2016.csv", index_col=0)

In [34]:
price_data.head()

,date,ticker,adjclose
2016-01-04,2016-01-04,DDD,9.83
2016-01-05,2016-01-05,DDD,9.69
2016-01-06,2016-01-06,DDD,9.10
2016-01-07,2016-01-07,DDD,8.58
2016-01-08,2016-01-08,DDD,8.37


In [59]:
len(rel_funds)

10544

In [64]:
prices = []

n = 0
m = 0

for d, t in zip(rel_funds.date[1000:], rel_funds.ticker[1000:]):
    try:
        price = price_data[(price_data.ticker == t) & (price_data.date == d)]["adjclose"].item()
        prices.append(price)
        n += 1
        print("n: ", str(n))
    except:
        prices.append(np.nan)
        m += 1
        print("m: ", str(m))

n:  1
n:  2
n:  3
n:  4
n:  5
n:  6
n:  7
n:  8
n:  9
n:  10
n:  11
n:  12
n:  13
n:  14
n:  15
n:  16
n:  17
m:  1
n:  18
n:  19
n:  20
n:  21
n:  22
n:  23
n:  24
n:  25
n:  26
n:  27
n:  28
n:  29
n:  30
n:  31
n:  32
n:  33
n:  34
n:  35
n:  36
n:  37
n:  38
n:  39
n:  40
m:  2
m:  3
m:  4
m:  5
n:  41
n:  42
n:  43
n:  44
n:  45
n:  46
n:  47
n:  48
n:  49
n:  50
n:  51
n:  52
n:  53
n:  54
n:  55
n:  56
n:  57
n:  58
n:  59
n:  60
n:  61
m:  6
m:  7
n:  62
n:  63
n:  64
n:  65
n:  66
n:  67
n:  68
n:  69
n:  70
n:  71
n:  72
n:  73
n:  74
n:  75
n:  76
n:  77
n:  78
n:  79
n:  80
n:  81
n:  82
n:  83
n:  84
n:  85
n:  86
n:  87
n:  88
n:  89
n:  90
n:  91
n:  92
n:  93
n:  94
n:  95
n:  96
n:  97
n:  98
n:  99
n:  100
n:  101
n:  102
n:  103
n:  104
n:  105
n:  106
n:  107
n:  108
n:  109
n:  110
n:  111
n:  112
n:  113
n:  114
n:  115
n:  116
n:  117
n:  118
n:  119
n:  120
n:  121
n:  122
n:  123
n:  124
m:  8
m:  9
m:  10
n:  125
n:  126
n:  127
n:  128
n:  129
n:  130
n:  131

In [63]:
prices_to_1000 = prices

In [66]:
# add them to the 1000 we already have
for i in prices:
    prices_to_1000.append(i)

In [68]:
len(prices_to_1000)

10544

In [71]:
rel_funds["adj_close"] = prices_to_1000

C:\Users\Leonhard\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [72]:
rel_funds.head(20)

,date,ticker,assets,book_equity,shares_rep,shares_outstanding,adj_close
0,2020-09-30,ARCC,1.495000e+10,6.963000e+09,NaN,422620000.0,13.950000
1,2020-06-30,ARCC,1.451700e+10,6.691000e+09,NaN,423000000.0,14.053294
2,2020-03-31,ARCC,1.580600e+10,6.583000e+09,-100000000.0,423000000.0,10.198769
3,2019-12-31,ARCC,1.490500e+10,7.467000e+09,-100000000.0,431000000.0,17.149157
4,2020-09-30,AEIS,1.642519e+09,7.681520e+08,-4331000.0,38270000.0,62.939999
5,2020-06-30,AEIS,1.594250e+09,7.178130e+08,-4331000.0,38320000.0,67.790001
6,2020-03-31,AEIS,1.522550e+09,6.853970e+08,-7248000.0,38250000.0,48.490002
7,2019-12-31,AEIS,1.532406e+09,6.767140e+08,-7248000.0,38360000.0,71.199997
8,2020-09-30,DOX,6.341621e+09,3.622646e+09,-90850000.0,131540000.0,57.410000
9,2020-06-30,DOX,6.384355e+09,3.604658e+09,-60014000.0,133020000.0,60.536541


In [79]:
# it is sad, there are a lot of prices missing due to the date
missing_data_df = rel_funds[rel_funds.adj_close.isnull()]
missing_data_df.reset_index(drop = True, inplace = True)

In [98]:
rel_funds.to_csv("2020-01-25_funds_data.csv")

In [99]:
# import datetime
import datetime

In [103]:
missing_data_df.date.unique()

array(['2020-11-29', '2020-08-30', '2020-05-31', '2020-02-23',
       '2020-09-26', '2020-06-27', '2020-03-28', '2019-12-28',
       '2020-10-31', '2020-09-27', '2020-06-28', '2020-03-29',
       '2019-12-29', '2020-12-31', '2020-06-30', '2019-12-31',
       '2019-09-30', '2020-10-03', '2020-08-01', '2020-05-02',
       '2020-10-25', '2020-07-26', '2020-04-26', '2020-01-26',
       '2020-11-01', '2020-08-02', '2020-05-03', '2020-02-02',
       '2020-02-01', '2020-07-03', '2020-11-28', '2020-08-29',
       '2020-05-30', '2020-02-29', '2020-11-30', '2019-12-25',
       '2020-09-30', '2020-03-31', '2020-10-04', '2020-07-05',
       '2020-04-05', '2019-11-30', '2020-11-22', '2020-05-10',
       '2020-02-16', '2020-07-04', '2020-04-04', '2020-01-04',
       '2020-10-24', '2020-07-25', '2020-04-25', '2020-01-25',
       '2020-09-05', '2020-06-13', '2020-03-21', '2021-01-02',
       '2019-09-29', '2020-12-26', '2020-11-07', '2020-08-15',
       '2020-05-23', '2020-07-12', '2020-04-19', '2020-

## Scraping new stock price data

In [116]:
len(price_data.ticker.unique())


6615

In [117]:
ticker_df_list = []
n = 0
m = 0
for ticker in price_data.ticker.unique():
    
    try:
        ticker_df = si.get_data(ticker, start_date="2020-11-21")[["ticker", "adjclose"]]
        ticker_df.insert(0, "date", ticker_df.index)
        ticker_df_list.append(ticker_df)
        n += 1
        print("n: ", str(n))
    except:
        m += 1
        print("m: ", str(m))
        pass
        


n:  1
n:  2
n:  3
n:  4
n:  5
n:  6
n:  7
n:  8
n:  9
n:  10
n:  11
n:  12
n:  13
n:  14
n:  15
n:  16
n:  17
n:  18
n:  19
n:  20
n:  21
n:  22
n:  23
n:  24
n:  25
n:  26
n:  27
n:  28
n:  29
n:  30
n:  31
n:  32
n:  33
n:  34
n:  35
n:  36
n:  37
n:  38
n:  39
n:  40
n:  41
n:  42
n:  43
n:  44
n:  45
n:  46
n:  47
n:  48
n:  49
n:  50
n:  51
n:  52
n:  53
n:  54
n:  55
n:  56
n:  57
n:  58
n:  59
n:  60
n:  61
n:  62
n:  63
n:  64
n:  65
n:  66
n:  67
n:  68
n:  69
n:  70
n:  71
n:  72
n:  73
n:  74
n:  75
n:  76
n:  77
n:  78
n:  79
n:  80
n:  81
n:  82
n:  83
n:  84
n:  85
n:  86
n:  87
n:  88
n:  89
n:  90
n:  91
n:  92
n:  93
n:  94
n:  95
n:  96
n:  97
n:  98
n:  99
n:  100
n:  101
n:  102
n:  103
n:  104
n:  105
n:  106
n:  107
n:  108
n:  109
n:  110
n:  111
n:  112
n:  113
n:  114
n:  115
n:  116
n:  117
n:  118
n:  119
n:  120
n:  121
n:  122
n:  123
n:  124
n:  125
n:  126
n:  127
n:  128
n:  129
n:  130
n:  131
n:  132
n:  133
n:  134
n:  135
n:  136
n:  137
n:  138
n:  

In [136]:
latest_price_date = pd.concat(ticker_df_list)
latest_price_date.shape


(268862, 3)

In [137]:
latest_price_date["date"] = [date.strftime("%Y-%m-%d") for date in latest_price_date.date]

In [142]:
all_price_data = price_data.append(latest_price_date)

In [143]:
prices = []

n = 0
m = 0

for d, t in zip(missing_data_df.date, missing_data_df.ticker):
    try:
        price = all_price_data[(all_price_data.ticker == t) & (all_price_data.date == d)]["adjclose"].item()
        prices.append(price)
        n += 1
        print("n: ", str(n))
    except:
        prices.append(np.nan)
        m += 1
        print("m: ", str(m))

m:  1
m:  2
m:  3
m:  4
m:  5
m:  6
m:  7
m:  8
m:  9
m:  10
m:  11
m:  12
m:  13
m:  14
n:  1
m:  15
m:  16
m:  17
m:  18
m:  19
m:  20
m:  21
m:  22
m:  23
m:  24
m:  25
m:  26
m:  27
m:  28
m:  29
m:  30
m:  31
m:  32
m:  33
m:  34
m:  35
m:  36
m:  37
m:  38
m:  39
m:  40
m:  41
m:  42
m:  43
m:  44
m:  45
m:  46
m:  47
m:  48
m:  49
m:  50
m:  51
m:  52
m:  53
m:  54
m:  55
n:  2
m:  56
m:  57
n:  3
m:  58
m:  59
m:  60
m:  61
m:  62
m:  63
m:  64
m:  65
m:  66
m:  67
m:  68
m:  69
n:  4
m:  70
m:  71
n:  5
n:  6
m:  72
m:  73
m:  74
m:  75
m:  76
m:  77
m:  78
n:  7
m:  79
m:  80
m:  81
m:  82
m:  83
m:  84
m:  85
m:  86
m:  87
m:  88
m:  89
m:  90
m:  91
m:  92
m:  93
m:  94
m:  95
m:  96
m:  97
m:  98
m:  99
m:  100
m:  101
m:  102
m:  103
m:  104
m:  105
m:  106
m:  107
m:  108
n:  8
n:  9
n:  10
n:  11
m:  109
m:  110
m:  111
m:  112
m:  113
m:  114
m:  115
m:  116
m:  117
m:  118
m:  119
m:  120
m:  121
m:  122
m:  123
m:  124
m:  125
m:  126
m:  127
m:  128
m:  129
m:  130


In [144]:
missing_data_df["adj_close"] = prices

C:\Users\Leonhard\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [152]:
rel_funds = rel_funds.dropna(subset=["adj_close"]).append(missing_data_df)

In [155]:
missing_data_df = rel_funds[rel_funds.adj_close.isnull()]

In [159]:
test_df = all_price_data[all_price_data.ticker == "MSFT"] # microsoft price data as our test df

# define function that corrects date
def get_correct_date(date, test_df):
    day = int(date[-2:])
    month = int(date[-5:-3])
    year = int(date[:4])
    while len(test_df[test_df.date == date]) == 0:
        if month == 12 and day == 31:
            year += 1
            day = 1
            month = 1
            date = datetime.date(year, month, day)
            date = date.strftime("%Y-%m-%d")

        else:
            try:
                day += 1
                date = datetime.date(year, month, day)
                date = date.strftime("%Y-%m-%d")

            except:
                month += 1
                day = 1
                date = datetime.date(year, month, day)
                date = date.strftime("%Y-%m-%d")
    return date

In [161]:
# apply it

correct_dates = [get_correct_date(d, test_df) for d in missing_data_df.date]

missing_data_df["alt_date"] = correct_dates
    
    


C:\Users\Leonhard\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [163]:
prices = []

n = 0
m = 0

for d, t in zip(missing_data_df.alt_date, missing_data_df.ticker):
    try:
        price = all_price_data[(all_price_data.ticker == t) & (all_price_data.date == d)]["adjclose"].item()
        prices.append(price)
        n += 1
        print("n: ", str(n))
    except:
        prices.append(np.nan)
        m += 1
        print("m: ", str(m))

n:  1
n:  2
n:  3
n:  4
n:  5
n:  6
n:  7
n:  8
n:  9
n:  10
n:  11
n:  12
n:  13
n:  14
n:  15
m:  1
m:  2
m:  3
n:  16
n:  17
n:  18
n:  19
n:  20
n:  21
n:  22
n:  23
n:  24
n:  25
n:  26
n:  27
n:  28
n:  29
n:  30
n:  31
n:  32
n:  33
n:  34
n:  35
n:  36
n:  37
n:  38
n:  39
n:  40
n:  41
n:  42
n:  43
n:  44
n:  45
n:  46
n:  47
n:  48
n:  49
n:  50
n:  51
n:  52
n:  53
n:  54
n:  55
n:  56
n:  57
n:  58
n:  59
n:  60
n:  61
n:  62
n:  63
n:  64
n:  65
n:  66
n:  67
n:  68
n:  69
n:  70
n:  71
m:  4
n:  72
n:  73
n:  74
n:  75
n:  76
n:  77
n:  78
n:  79
n:  80
n:  81
n:  82
n:  83
n:  84
n:  85
n:  86
n:  87
n:  88
n:  89
n:  90
n:  91
n:  92
n:  93
n:  94
n:  95
n:  96
n:  97
n:  98
n:  99
n:  100
n:  101
n:  102
n:  103
n:  104
n:  105
n:  106
n:  107
n:  108
n:  109
n:  110
n:  111
n:  112
n:  113
n:  114
n:  115
n:  116
n:  117
n:  118
n:  119
n:  120
n:  121
n:  122
n:  123
n:  124
n:  125
n:  126
n:  127
n:  128
n:  129
n:  130
n:  131
n:  132
n:  133
n:  134
n:  135
n:  

In [165]:
missing_data_df["adj_close"] = prices

C:\Users\Leonhard\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [168]:
corrected_miss_data = missing_data_df.dropna(subset=["adj_close"]).drop(columns = {"alt_date"})

In [170]:
rel_funds = rel_funds.dropna(subset=["adj_close"]).append(corrected_miss_data)

In [172]:
rel_funds["market_cap"] = rel_funds["shares_outstanding"] * rel_funds["adj_close"]

In [173]:
rel_funds

,date,ticker,assets,book_equity,shares_rep,shares_outstanding,adj_close,market_cap
0,2020-09-30,ARCC,1.495000e+10,6.963000e+09,NaN,422620000.0,13.950000,5.895549e+09
1,2020-06-30,ARCC,1.451700e+10,6.691000e+09,NaN,423000000.0,14.053294,5.944543e+09
2,2020-03-31,ARCC,1.580600e+10,6.583000e+09,-100000000.0,423000000.0,10.198769,4.314079e+09
3,2019-12-31,ARCC,1.490500e+10,7.467000e+09,-100000000.0,431000000.0,17.149157,7.391286e+09
4,2020-09-30,AEIS,1.642519e+09,7.681520e+08,-4331000.0,38270000.0,62.939999,2.408714e+09
...,...,...,...,...,...,...,...,...
1432,2020-09-26,WINA,3.583860e+07,-8.833700e+06,NaN,3735000.0,165.223709,6.171106e+08
1433,2020-06-27,WINA,3.162640e+07,-1.856620e+07,NaN,3723000.0,164.978683,6.142156e+08
1434,2020-03-28,WINA,5.989180e+07,-2.984820e+07,-48987500.0,3651000.0,127.405624,4.651579e+08
1435,2019-12-28,WINA,6.184220e+07,1.244830e+07,-48987500.0,3948000.0,193.356934,7.633732e+08


In [248]:
# actually, i forgot the ebit on the way...
# sort by assets
all_funds_df = all_funds_df.sort_values(by = ["ticker", "ebit_date"])
all_funds_df.reset_index(drop = True, inplace = True)
all_funds_df

,Unnamed: 0,ticker,ebit_date,ebit,ass_tot_date,ass_tot,book_equ_date,book_equ,share_rep_date,share_rep
0,3,A,2020-01-31,230000000,2020-01-31,9.501000e+09,2020-01-31,4.848000e+09,2020-01-31,-93000000.0
1,2,A,2020-04-30,227000000,2020-04-30,9.455000e+09,2020-04-30,4.768000e+09,2020-04-30,-126000000.0
2,1,A,2020-07-31,254000000,2020-07-31,9.546000e+09,2020-07-31,4.981000e+09,2020-07-31,-34000000.0
3,0,A,2020-10-31,320000000,2020-10-31,9.627000e+09,2020-10-31,4.873000e+09,2020-10-31,-253000000.0
4,3,AAL,2019-12-31,923000000,2019-12-31,5.999500e+10,2019-12-31,-1.180000e+08,2019-12-31,-272000000.0
...,...,...,...,...,...,...,...,...,...,...
11339,0,ZUO,2020-10-31,-15042000,2020-10-31,3.953340e+08,2020-10-31,1.682910e+08,2020-10-31,-7000.0
11340,3,ZVO,2019-12-31,-7749000,2019-12-31,2.501380e+08,2019-12-31,9.893800e+07,2019-12-31,-9000.0
11341,2,ZVO,2020-03-31,-7732000,2020-03-31,2.618680e+08,2020-03-31,1.049850e+08,2020-03-31,-202000.0
11342,1,ZVO,2020-06-30,5768000,2020-06-30,2.641200e+08,2020-06-30,1.120050e+08,2020-06-30,-286000.0


In [202]:
rel_funds = rel_funds.sort_values(by = ["ticker", "date"])
rel_funds.reset_index(drop = True, inplace = True)
rel_funds

,date,ticker,assets,book_equity,shares_rep,shares_outstanding,adj_close,market_cap
0,2020-01-31,A,9.501000e+09,4.848000e+09,-93000000.0,310000000.0,82.029961,2.542929e+10
1,2020-04-30,A,9.455000e+09,4.768000e+09,-126000000.0,308000000.0,76.361679,2.351940e+10
2,2020-07-31,A,9.546000e+09,4.981000e+09,-34000000.0,309000000.0,96.156624,2.971240e+10
3,2020-10-31,A,9.627000e+09,4.873000e+09,-253000000.0,306000000.0,105.239998,3.220344e+10
4,2019-12-31,AAL,5.999500e+10,-1.180000e+08,-272000000.0,428200000.0,28.574404,1.223556e+10
...,...,...,...,...,...,...,...,...
10473,2020-10-31,ZUO,3.953340e+08,1.682910e+08,-7000.0,119000000.0,9.510000,1.131690e+09
10474,2019-12-31,ZVO,2.501380e+08,9.893800e+07,-9000.0,30330000.0,2.060000,6.247980e+07
10475,2020-03-31,ZVO,2.618680e+08,1.049850e+08,-202000.0,30660000.0,1.650000,5.058900e+07
10476,2020-06-30,ZVO,2.641200e+08,1.120050e+08,-286000.0,32140000.0,3.060000,9.834840e+07


In [239]:
rel_funds = rel_funds.drop_duplicates()

In [249]:
n = 0
ebit = []

for d, t in zip(rel_funds.date, rel_funds.ticker):
    try:
        e = all_funds_df[(all_funds_df.ebit_date == d) & (all_funds_df.ticker == t)]["ebit"].item()
        ebit.append(e)
        n += 1
    except:
        ebit.append(np.nan)
        print(n)
        n += 1

890
891
6104


In [250]:
rel_funds["ebit"] = ebit

C:\Users\Leonhard\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [252]:
rel_funds.reset_index(drop = True, inplace = True)

In [261]:

ticker = random.choice(rel_funds.ticker)
funds[funds.ticker == ticker]

,Unnamed: 0,ticker,ebit_date,ebit,ass_tot_date,ass_tot,book_equ_date,book_equ,share_rep_date,share_rep
3344,3,FARO,2019-12-31,-195000,2019-12-31,486842000.0,2019-12-31,331992000.0,2019-12-31,-810000.0
3345,2,FARO,2020-03-31,-2866000,2020-03-31,470049000.0,2020-03-31,316790000.0,2020-03-31,-1581000.0
3346,1,FARO,2020-06-30,-11326000,2020-06-30,453307000.0,2020-06-30,308561000.0,2020-06-30,-828000.0
3347,0,FARO,2020-09-30,-4619000,2020-09-30,451726000.0,2020-09-30,313330000.0,2020-09-30,-159000.0


In [262]:
rel_funds[rel_funds.ticker == ticker]

,date,ticker,assets,book_equity,shares_rep,shares_outstanding,adj_close,market_cap,ebit
3339,2019-12-31,FARO,486842000.0,331992000.0,-810000.0,17580000.0,50.349998,8.851530e+08,-195000.0
3340,2020-03-31,FARO,470049000.0,316790000.0,-1581000.0,17720000.0,44.500000,7.885400e+08,-2866000.0
3341,2020-06-30,FARO,453307000.0,308561000.0,-828000.0,17770000.0,53.599998,9.524720e+08,-11326000.0
3342,2020-09-30,FARO,451726000.0,313330000.0,-159000.0,17830000.0,60.980000,1.087273e+09,-4619000.0


In [263]:
rel_funds.to_csv("../2020-01-25_funds_df.csv")

In [267]:
all_price_data.columns = ["date", "ticker", "adj_close"]

In [269]:
all_price_data.to_csv("../relevant_price_data_from_2016.csv")

In [270]:
all_price_data.head()

,date,ticker,adj_close
2016-01-04,2016-01-04,DDD,9.83
2016-01-05,2016-01-05,DDD,9.69
2016-01-06,2016-01-06,DDD,9.10
2016-01-07,2016-01-07,DDD,8.58
2016-01-08,2016-01-08,DDD,8.37


In [276]:
rel_funds.sort_values(by = "date")[40:100]

,date,ticker,assets,book_equity,shares_rep,shares_outstanding,adj_close,market_cap,ebit
3620,2019-12-27,FN,1.315028e+09,9.283270e+08,-233000.0,3.702000e+07,64.570000,2.390381e+09,3.208000e+07
3924,2019-12-27,GGG,1.692210e+09,1.024931e+09,-4361000.0,1.672900e+08,51.501240,8.615642e+09,1.053150e+08
6153,2019-12-27,MRCY,1.512620e+09,1.317056e+09,-375000.0,5.456000e+07,70.300003,3.835568e+09,2.365000e+07
4276,2019-12-27,HCKT,1.937350e+08,1.376140e+08,-2276000.0,2.976000e+07,15.652542,4.658197e+08,8.371000e+06
708,2019-12-27,ARMK,1.427306e+10,3.451576e+09,-6540000.0,2.500600e+08,43.000401,1.075268e+10,2.542930e+08
5088,2019-12-27,JOUT,4.861230e+08,3.302390e+08,-460000.0,1.007000e+07,75.251701,7.577846e+08,6.719000e+06
921,2019-12-27,AVNW,1.798190e+08,6.846700e+07,-653000.0,5.414000e+06,14.120000,7.644568e+07,-1.116000e+06
4668,2019-12-27,ICHR,5.665550e+08,2.214160e+08,-68000.0,2.262000e+07,33.849998,7.656870e+08,5.006000e+06
2744,2019-12-27,DLB,2.893284e+09,2.333290e+09,-50285000.0,1.008800e+08,67.643173,6.823843e+09,4.926500e+07
9430,2019-12-27,TWIN,3.432810e+08,1.689240e+08,-224000.0,1.341000e+07,10.420000,1.397322e+08,-7.020000e+05


In [313]:
tsla = all_price_data[all_price_data.ticker == "TSLA"]

date_price = tsla[tsla.date == "2019-11-01"][:1]["adj_close"].item()
today = tsla[tsla.date == "2020-10-01"][:1]["adj_close"].item()

mom = (today - date_price)/date_price
print(mom)

6.152022160334295


In [291]:

tsla

,date,ticker,adj_close
2016-01-04,2016-01-04,TSLA,44.681999
2016-01-05,2016-01-05,TSLA,44.686001
2016-01-06,2016-01-06,TSLA,43.807999
2016-01-07,2016-01-07,TSLA,43.130001
2016-01-08,2016-01-08,TSLA,42.200001
...,...,...,...
2021-01-19 00:00:00,2021-01-19,TSLA,844.549988
2021-01-20 00:00:00,2021-01-20,TSLA,850.450012
2021-01-21 00:00:00,2021-01-21,TSLA,844.989990
2021-01-22 00:00:00,2021-01-22,TSLA,846.640015
